In [35]:
%load_ext autoreload
%autoreload 2

import sys  
sys.path.insert(1, '../')
sys.path.insert(1, '/home/avijendra/nl2sql_lib/nl2sql-generic/nl2sql_src')
print(sys.path)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
['/home/avijendra/nl2sql_lib/nl2sql-generic/notebooks', '/home/avijendra/nl2sql_lib/nl2sql-generic/nl2sql_src', '../', '/home/avijendra/nl2sql_lib/nl2sql-generic/nl2sql_src', '../', '/home/avijendra/nl2sql_lib/nl2sql-generic/nl2sql_src', '../', '/usr/lib/python311.zip', '/usr/lib/python3.11', '/usr/lib/python3.11/lib-dynload', '', '/home/avijendra/fserv/lib/python3.11/site-packages']


In [36]:
from nl2sql_generic import Nl2sqlBq
PROJECT_ID = 'sl-test-project-363109'
LOCATION = 'us-central1'
DATASET_ID = 'zoominfo'

In [37]:
# Initializing 1st time for a dataset
# nl2sqlbq_client = Nl2sqlBq(project_id=PROJECT_ID, dataset_id=DATASET_ID)

# When Initializing for first time , you will need to create metadata_json
# nl2sqlbq_client.create_metadata_json(metadata_json_dest_path = "../nl2sql_src/cache_metadata/metadata_cache.json",data_dict_path="../nl2sql_src/data_dict/data_dictionary.json",col_values_distribution = True) #data_dict_path is optional . It is used to provide descriptions for tables and columns

In [38]:
# Initializing when metadata cache is already created
nl2sqlbq_client = Nl2sqlBq(project_id=PROJECT_ID, dataset_id=DATASET_ID, metadata_json_path = "../nl2sql_src/cache_metadata/metadata_cache.json",model_name="text-bison@002", tuned_model=True)
nl2sqlbq_client_base = Nl2sqlBq(project_id=PROJECT_ID, dataset_id=DATASET_ID, metadata_json_path = "../nl2sql_src/cache_metadata/metadata_cache.json",model_name="text-bison@002", tuned_model=False)

text-bison@002
text-bison@002
text-bison@002
text-bison@002


In [39]:
# question = "What is the total revenue for the construction primary industry ?"
question = "What are the top 5 industries in terms of revenue?"

In [40]:
# Convert to sql
# table_name = nl2sqlbq_client.table_filter(question) 
query_base = nl2sqlbq_client_base.generate_sql(question) # Optionally provide table name for faster exection by avoiding table filtering when there are more than 1 table in dataset
print(query_base)
query = nl2sqlbq_client.generate_sql(question) # Optionally provide table name for faster exection by avoiding table filtering when there are more than 1 table in dataset
# query = query.split(':')[1].strip()
print(query)

Originall query =   
SELECT 
  ZI_ES_INDUSTRY AS industry,
  SUM(ZI_C_REVENUE) AS total_revenue
FROM 
  `zi-retail-companies-headquartered-california`
GROUP BY 
  industry
ORDER BY 
  total_revenue DESC
LIMIT 5;

 
SELECT 
  ZI_ES_INDUSTRY AS industry,
  SUM(ZI_C_REVENUE) AS total_revenue
FROM sl-test-project-363109.zoominfo.`zi-retail-companies-headquartered-california`
GROUP BY 
  industry
ORDER BY 
  total_revenue DESC
LIMIT 5;

Originall query =   Response: SELECT ZI_ES_INDUSTRY, SUM(ZI_C_REVENUE) FROM zi-retail-companies-headquartered-california GROUP BY ZI_ES_INDUSTRY ORDER BY SUM(ZI_C_REVENUE) DESC LIMIT 5
SELECT ZI_ES_INDUSTRY, SUM(ZI_C_REVENUE) FROM sl-test-project-363109.zoominfo.`zi-retail-companies-headquartered-california` GROUP BY ZI_ES_INDUSTRY ORDER BY SUM(ZI_C_REVENUE) DESC LIMIT 5


In [41]:
# Execute query
nl2sqlbq_client.execute_query(query_base)

,industry,total_revenue
0,Retail,281964937
1,Consumer Electronics,86153757
2,Automotive,38272006
3,Apparel & Fashion,16427778
4,Consumer Goods,5971238


In [42]:
nl2sqlbq_client.execute_query(query)

,ZI_ES_INDUSTRY,f0_
0,Retail,281964937
1,Consumer Electronics,86153757
2,Automotive,38272006
3,Apparel & Fashion,16427778
4,Consumer Goods,5971238


In [43]:
#Convert to sql and excute
nl2sqlbq_client.text_to_sql_execute(question)

Originall query =   Response: SELECT ZI_ES_INDUSTRY, SUM(ZI_C_REVENUE) FROM zi-retail-companies-headquartered-california GROUP BY ZI_ES_INDUSTRY ORDER BY SUM(ZI_C_REVENUE) DESC LIMIT 5
SELECT ZI_ES_INDUSTRY, SUM(ZI_C_REVENUE) FROM sl-test-project-363109.zoominfo.`zi-retail-companies-headquartered-california` GROUP BY ZI_ES_INDUSTRY ORDER BY SUM(ZI_C_REVENUE) DESC LIMIT 5


,ZI_ES_INDUSTRY,f0_
0,Retail,281964937
1,Consumer Electronics,86153757
2,Automotive,38272006
3,Apparel & Fashion,16427778
4,Consumer Goods,5971238


In [44]:
# If you already know table_name provide it, for more accurate and faster execution by avoiding table_filtering when there are more than 1 table in dataset
sql_execution_response = nl2sqlbq_client.text_to_sql_execute(question, table_name="zi-retail-companies-headquartered-california")
print(sql_execution_response)

Originall query =   Response: SELECT ZI_ES_INDUSTRY, SUM(ZI_C_REVENUE) FROM zi-retail-companies-headquartered-california GROUP BY ZI_ES_INDUSTRY ORDER BY SUM(ZI_C_REVENUE) DESC LIMIT 5
SELECT ZI_ES_INDUSTRY, SUM(ZI_C_REVENUE) FROM sl-test-project-363109.zoominfo.`zi-retail-companies-headquartered-california` GROUP BY ZI_ES_INDUSTRY ORDER BY SUM(ZI_C_REVENUE) DESC LIMIT 5
         ZI_ES_INDUSTRY        f0_
0                Retail  281964937
1  Consumer Electronics   86153757
2            Automotive   38272006
3     Apparel & Fashion   16427778
4        Consumer Goods    5971238


In [45]:
# To convert reponse in form of natural language use result2nl
nl2sqlbq_client.result2nl(result = sql_execution_response, question = question)

' Response: Retail industry is the highest revenue generating industry with a revenue of $281,964,937. Consumer Electronics industry is the second highest revenue generating industry with a revenue of $86,153,757. Automotive industry is the third highest revenue generating industry with a revenue of $38,272,006. Apparel & Fashion industry is the fourth highest revenue generating industry with a revenue of $16,427,778. Consumer Goods industry is the fifth highest revenue generating industry with a revenue of $5,971,238.'

In [46]:
# For testing batch of questions with ground_truth sql
# nl2sqlbq_client.batch_run(test_file_name = "/test_file-1.csv", output_file_name="/test-file-1-output.csv")

In [47]:
# For testing batch of questions with full functionality
# nl2sqlbq_client.batch_run(test_file_name = "/test_file-1.csv", output_file_name="/test-file-1-output.csv",execute_query = True, result2nl=True, logger_file = "log_batch.txt" )

## Few Shot Prompting

In [48]:
PGPROJ = "sl-test-project-353312"
PGLOCATION = 'us-central1'
PGINSTANCE = "test-nl2sql"
PGDB = "test-db"
PGTABLE = 'myqueries'
PGUSER = "postgres"
PGPWD = "test-nl2sql"
nl2sqlbq_client.init_pgdb(PGPROJ, PGLOCATION, PGINSTANCE, PGDB, PGUSER, PGPWD, pg_table=PGTABLE)

PGSQL - class Initiated


In [49]:
query = nl2sqlbq_client.generate_sql_few_shot(question) # Optionally provide table name for faster exection by avoiding table filtering when there are more than 1 table in dataset
print(query)

Trying to read the index file ../../nl2sql-generic/nl2sql_src/cache_metadata/saved_index_pgdata
Originall query =   Response: select ZI_C_INDUSTRY_PRIMARY from `zoominfo.zi-retail-companies-headquartered-california` order by ZI_C_REVENUE desc limit 5
select ZI_C_INDUSTRY_PRIMARY FROM sl-test-project-363109.zoominfo.`zoominfo.zi-retail-companies-headquartered-california` order by ZI_C_REVENUE desc limit 5


#### Verify if the query is valid by performing a dry run on BQ

In [50]:
# Execute query - dry run only
query_status, message = nl2sqlbq_client.execute_query(query, dry_run=True)
print(message, query_status)

BadRequest: 400 POST https://bigquery.googleapis.com/bigquery/v2/projects/cdii-poc/jobs?prettyPrint=false: Invalid project ID 'sl-test-project-363109.zoominfo'. Project IDs must contain 6-63 lowercase letters, digits, or dashes. Some project IDs also include domain name separated by a colon. IDs must start with a letter and may not end with a dash.

Location: None
Job ID: 1c45073e-a806-4d9e-9c44-431776baa830


#### Self-reflection if the query is invalid

In [51]:
# Regenerate the query checking for validity. Max-tries = 5
query_status = False
if not query_status:
    query_status, query = nl2sqlbq_client.self_reflection(question=question, query=query) 

BadRequest: 400 POST https://bigquery.googleapis.com/bigquery/v2/projects/cdii-poc/jobs?prettyPrint=false: Invalid project ID 'sl-test-project-363109.zoominfo'. Project IDs must contain 6-63 lowercase letters, digits, or dashes. Some project IDs also include domain name separated by a colon. IDs must start with a letter and may not end with a dash.

Location: None
Job ID: 8a1a9139-c92a-4b4a-bfca-d9358ef9487c


In [ ]:
result = nl2sqlbq_client.execute_query(query)
nl2sqlbq_client.result2nl(result=result, question=question)